# Título: "Modelo MLP para clasificación usando características CWT promediadas"

# 1. Configuración inicial y carga de datos

In [ ]:
# Importar bibliotecas necesarias
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import h5py
import json
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras import backend as K
import keras_tuner as kt
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Función para cargar el dataset de características promediadas
def load_cwt_averaged_dataset(file_path):
    """
    Carga el dataset de características CWT promediadas desde archivo H5
    """
    with h5py.File(file_path, 'r') as hf:
        # Cargar características y etiquetas
        x_train = np.array(hf['train']['features'])
        y_train = np.array(hf['train']['labels_onehot'])
        y_train_raw = np.array(hf['train']['labels'])
        
        x_test = np.array(hf['test']['features'])
        y_test = np.array(hf['test']['labels_onehot'])
        y_test_raw = np.array(hf['test']['labels'])
        
        # Cargar metadatos
        num_classes = hf['metadata']['num_classes'][()]
        num_features = hf['metadata']['num_features'][()]
        
        # Cargar diccionario de etiquetas
        label_codes_dict = json.loads(hf['metadata'].attrs['label_codes_dict'])
    
    print(f"Dataset cargado exitosamente:")
    print(f"- Características de entrenamiento: {x_train.shape}")
    print(f"- Características de prueba: {x_test.shape}")
    print(f"- Número de clases: {num_classes}")
    print(f"- Dimensiones por muestra: {num_features}")
    
    # Visualizar distribución de clases
    plt.figure(figsize=(10, 5))
    class_counts = np.sum(y_train, axis=0)
    sns.barplot(x=list(label_codes_dict.keys()), y=class_counts)
    plt.title("Distribución de clases en conjunto de entrenamiento")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return {
        'x_train': x_train,
        'y_train': y_train,
        'y_train_raw': y_train_raw,
        'x_test': x_test,
        'y_test': y_test,
        'y_test_raw': y_test_raw,
        'num_classes': num_classes,
        'num_features': num_features,
        'label_codes_dict': label_codes_dict
    }

# Cargar dataset
dataset_path = '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/accelerometer_cwt_averaged_ns512_processed.h5'
dataset = load_cwt_averaged_dataset(dataset_path)

# Dividir datos para tener un conjunto de validación separado
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    dataset['x_train'], dataset['y_train'], 
    test_size=0.2, stratify=dataset['y_train_raw'], random_state=42
)

print(f"Conjunto de entrenamiento: {x_train.shape}, {y_train.shape}")
print(f"Conjunto de validación: {x_val.shape}, {y_val.shape}")
print(f"Conjunto de prueba: {dataset['x_test'].shape}, {dataset['y_test'].shape}")

# 2. Modelo MLP básico de referencia


In [ ]:
# Función para crear un modelo MLP básico como referencia
def create_base_mlp(input_dim, num_classes):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Crear y entrenar modelo base
base_model = create_base_mlp(dataset['num_features'], dataset['num_classes'])
base_model.summary()

# Entrenar modelo base
base_history = base_model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )
    ]
)

# Evaluar modelo base
base_test_loss, base_test_acc = base_model.evaluate(dataset['x_test'], dataset['y_test'])
print(f"Precisión del modelo base en datos de prueba: {base_test_acc:.4f}")

# Visualizar curvas de aprendizaje
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(base_history.history['accuracy'], label='Entrenamiento')
plt.plot(base_history.history['val_accuracy'], label='Validación')
plt.title('Precisión del modelo base')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(base_history.history['loss'], label='Entrenamiento')
plt.plot(base_history.history['val_loss'], label='Validación')
plt.title('Pérdida del modelo base')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.tight_layout()
plt.show()

#  Modelo MLP con regularización avanzada

In [ ]:
def create_regularized_mlp(input_dim, num_classes):
    """
    Crea un modelo MLP con técnicas de regularización del estado del arte
    """
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        
        # Primera capa con regularización L2
        layers.Dense(128, 
                    kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Segunda capa con regularización L2
        layers.Dense(64, 
                    kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.3),
        
        # Tercera capa
        layers.Dense(32),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.2),
        
        # Capa de salida
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Learning rate con decay
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=1000,
        decay_rate=0.9
    )
    
    # Gradient clipping
    optimizer = keras.optimizers.Adam(
        learning_rate=lr_schedule,
        clipnorm=1.0  # Gradient clipping
    )
    
    # Label smoothing
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    
    return model

# Crear y entrenar modelo regularizado
reg_model = create_regularized_mlp(dataset['num_features'], dataset['num_classes'])
reg_model.summary()

# Callbacks para entrenamiento
callbacks = [
    # Early stopping
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=30, 
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reducción de learning rate cuando se estanca
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        verbose=1
    ),
    
    # Checkpoint para guardar el mejor modelo
    keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/best_reg_mlp.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Entrenar modelo regularizado
reg_history = reg_model.fit(
    x_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=callbacks
)

# Evaluar modelo regularizado
reg_test_loss, reg_test_acc = reg_model.evaluate(dataset['x_test'], dataset['y_test'])
print(f"Precisión del modelo regularizado en datos de prueba: {reg_test_acc:.4f}")

# Comparar con modelo base
print(f"Mejora sobre el modelo base: {(reg_test_acc - base_test_acc)*100:.2f}%")

# 4. NAS multiobjetivo con Keras Tuner

In [ ]:
# Definir clase para búsqueda NAS multiobjetivo
class MLPHyperModel(kt.HyperModel):
    def __init__(self, input_dim, num_classes):
        self.input_dim = input_dim
        self.num_classes = num_classes
    
    def build(self, hp):
        """
        Construye un modelo MLP con hiperparámetros ajustables
        """
        model = keras.Sequential()
        model.add(layers.Input(shape=(self.input_dim,)))
        
        # Determinar número de capas (entre 2 y 4)
        n_layers = hp.Int('num_layers', 2, 4)
        
        # Configurar capas ocultas
        for i in range(n_layers):
            # Número de neuronas por capa
            units = hp.Int(f'units_{i}', 
                          min_value=16, 
                          max_value=128, 
                          step=16)
            
            # Regularización L2
            reg_rate = hp.Choice(f'reg_{i}', 
                                values=[0.0, 0.0001, 0.0005, 0.001])
            
            # Tasa de dropout
            dropout_rate = hp.Float(f'dropout_{i}',
                                   min_value=0.0,
                                   max_value=0.5,
                                   step=0.1)
            
            # Añadir capa Dense
            model.add(layers.Dense(
                units,
                kernel_regularizer=regularizers.l2(reg_rate)
            ))
            
            # Activación
            activation = hp.Choice(f'activation_{i}', 
                                  values=['relu', 'elu', 'selu'])
            model.add(layers.Activation(activation))
            
            # Batch normalization (opcional)
            if hp.Boolean(f'batch_norm_{i}'):
                model.add(layers.BatchNormalization())
            
            # Dropout (opcional)
            if dropout_rate > 0:
                model.add(layers.Dropout(dropout_rate))
        
        # Capa de salida
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        
        # Compilar modelo
        learning_rate = hp.Float('learning_rate', 
                               min_value=1e-4,
                               max_value=1e-2,
                               sampling='log')
        
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def fit(self, hp, model, x, y, validation_data, **kwargs):
        """
        Entrena el modelo con early stopping
        """
        batch_size = hp.Int('batch_size', 16, 128, step=16)
        
        return model.fit(
            x, y,
            validation_data=validation_data,
            batch_size=batch_size,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=25,
                    restore_best_weights=True
                )
            ],
            **kwargs
        )

# Función para obtener tamaño del modelo en KB
def get_model_size(model):
    """Retorna el tamaño del modelo en KB"""
    # Convertir a TFLite para obtener tamaño realista
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    return len(tflite_model) / 1024  # Tamaño en KB

# Implementar objetivos personalizados para Keras Tuner
class MultiObjectiveNAS(kt.Tuner):
    def run_trial(self, trial, x, y, validation_data, **kwargs):
        hp = trial.hyperparameters
        model = self.hypermodel.build(hp)
        
        # Entrenar el modelo
        history = self.hypermodel.fit(
            hp, model, x, y, 
            validation_data=validation_data,
            epochs=100,  # Máximo número de épocas
            **kwargs
        )
        
        # Calcular métricas de rendimiento
        val_accuracy = max(history.history['val_accuracy'])
        
        # Calcular tamaño del modelo
        model_size = get_model_size(model)
        
        # Devolver métricas para optimización multiobjetivo
        return {
            'val_accuracy': val_accuracy,
            'model_size': model_size,
            'val_loss': min(history.history['val_loss'])
        }

# Configurar y ejecutar la búsqueda
hypermodel = MLPHyperModel(
    input_dim=dataset['num_features'], 
    num_classes=dataset['num_classes']
)

tuner = kt.RandomSearch(
    hypermodel,
    objective=[
        kt.Objective('val_accuracy', direction='max'),
        kt.Objective('model_size', direction='min')
    ],
    max_trials=50,
    directory='/content/drive/MyDrive/Tesis/Accelerometer_Dataset/nas_results',
    project_name='mlp_multiobj_nas'
)

# Mostrar resumen de la búsqueda
tuner.search_space_summary()

# Ejecutar búsqueda
tuner.search(
    x_train, y_train,
    validation_data=(x_val, y_val),
    verbose=1
)

# Obtener los mejores modelos según diferentes criterios
best_accuracy_model = tuner.get_best_models(1, objective='val_accuracy')[0]
best_size_model = tuner.get_best_models(1, objective='model_size')[0]

# Obtener frente de Pareto
pareto_models = tuner.get_best_models(10)

# Visualizar resultados de la búsqueda
trials_df = tuner.results_summary(return_dataframe=True)
plt.figure(figsize=(10, 6))
plt.scatter(
    trials_df['model_size'], 
    trials_df['val_accuracy'], 
    alpha=0.7
)

# Marcar el mejor modelo para cada objetivo
plt.scatter(
    trials_df['model_size'].min(),
    trials_df.loc[trials_df['model_size'].argmin(), 'val_accuracy'],
    color='red', marker='*', s=200, label='Modelo más pequeño'
)
plt.scatter(
    trials_df.loc[trials_df['val_accuracy'].argmax(), 'model_size'],
    trials_df['val_accuracy'].max(),
    color='green', marker='*', s=200, label='Modelo más preciso'
)

plt.xlabel('Tamaño del modelo (KB)')
plt.ylabel('Precisión de validación')
plt.title('Frente de Pareto: Tamaño vs Precisión')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

# Seleccionar modelo óptimo basado en balance entre precisión y tamaño
# Usar técnica de normalización y ponderación
normalized_size = (trials_df['model_size'] - trials_df['model_size'].min()) / (trials_df['model_size'].max() - trials_df['model_size'].min())
normalized_acc = (trials_df['val_accuracy'] - trials_df['val_accuracy'].min()) / (trials_df['val_accuracy'].max() - trials_df['val_accuracy'].min())

# Ponderación: 60% precisión, 40% tamaño
weighted_score = 0.6 * normalized_acc - 0.4 * normalized_size
best_idx = weighted_score.argmax()

print(f"Modelo óptimo seleccionado:")
print(f"- Precisión: {trials_df.iloc[best_idx]['val_accuracy']:.4f}")
print(f"- Tamaño: {trials_df.iloc[best_idx]['model_size']:.2f} KB")

# Obtener y reentrenar el modelo óptimo
best_hp = tuner.get_best_hyperparameters(1, objective=weighted_score.name)[0]
optimal_model = hypermodel.build(best_hp)
optimal_model.summary()

# Reentrenar el modelo óptimo con todos los datos de entrenamiento
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=30, 
        restore_best_weights=True
    ),
    keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/best_nas_mlp.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]

optimal_history = optimal_model.fit(
    dataset['x_train'], dataset['y_train'],
    epochs=200,
    batch_size=int(best_hp.get('batch_size')),
    validation_split=0.2,
    callbacks=callbacks
)


# 5. Evaluación del modelo óptimo

In [ ]:
# Evaluar el modelo óptimo
optimal_test_loss, optimal_test_acc = optimal_model.evaluate(
    dataset['x_test'], dataset['y_test']
)
print(f"Precisión del modelo óptimo en datos de prueba: {optimal_test_acc:.4f}")

# Obtener predicciones
y_pred_proba = optimal_model.predict(dataset['x_test'])
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = dataset['y_test_raw']

# Matriz de confusión
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred)
class_names = list(dataset['label_codes_dict'].keys())

# Visualizar matriz de confusión normalizada
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
           xticklabels=class_names, yticklabels=class_names)
plt.title('Matriz de Confusión Normalizada')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.tight_layout()
plt.show()

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Curvas ROC (one-vs-rest para multiclase)
plt.figure(figsize=(12, 8))
for i, class_name in enumerate(class_names):
    # Convertir a one-vs-rest para esta clase
    y_true_binary = (y_true == i).astype(int)
    y_score = y_pred_proba[:, i]
    
    # Calcular curva ROC
    fpr, tpr, _ = roc_curve(y_true_binary, y_score)
    roc_auc = auc(fpr, tpr)
    
    # Graficar
    plt.plot(fpr, tpr, label=f'{class_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Azar')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curvas ROC para cada clase')
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# Visualizar ejemplos mal clasificados
incorrect_idx = np.where(y_pred != y_true)[0]
if len(incorrect_idx) > 0:
    plt.figure(figsize=(12, len(incorrect_idx) * 2))
    for i, idx in enumerate(incorrect_idx[:10]):  # Mostrar hasta 10 ejemplos incorrectos
        x_sample = dataset['x_test'][idx]
        true_label = class_names[y_true[idx]]
        pred_label = class_names[y_pred[idx]]
        
        plt.subplot(min(len(incorrect_idx), 10), 1, i+1)
        plt.plot(x_sample)
        plt.title(f'Verdadero: {true_label}, Predicho: {pred_label}')
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()


# 6. Exportación para STM32CubeAI

In [ ]:
# Guardar el modelo en formato TF
optimal_model.save('/content/drive/MyDrive/Tesis/Accelerometer_Dataset/optimal_mlp_model.h5')

# Función para convertir a TFLite con diferentes configuraciones
def convert_to_tflite(model, quantize=False, optimize=True):
    """
    Convierte el modelo a TFLite con diferentes opciones de optimización
    """
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    if optimize:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    
    if quantize:
        # Quantize to int8
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8
        converter.inference_output_type = tf.int8
        
        # Representative dataset generator 
        def representative_dataset():
            for i in range(min(100, len(dataset['x_train']))):  # Usar 100 muestras
                sample = dataset['x_train'][i:i+1].astype(np.float32)
                yield [sample]
                
        converter.representative_dataset = representative_dataset
    
    tflite_model = converter.convert()
    return tflite_model

# Convertir a diferentes formatos
print("Convirtiendo modelos para STM32CubeAI...")

# Modelo float32 (no optimizado)
tflite_float32 = convert_to_tflite(optimal_model, quantize=False, optimize=False)
with open('/content/drive/MyDrive/Tesis/Accelerometer_Dataset/optimal_mlp_float32.tflite', 'wb') as f:
    f.write(tflite_float32)
print(f"Modelo float32: {len(tflite_float32)/1024:.2f} KB")

# Modelo float32 optimizado
tflite_float32_optimized = convert_to_tflite(optimal_model, quantize=False, optimize=True)
with open('/content/drive/MyDrive/Tesis/Accelerometer_Dataset/optimal_mlp_float32_optimized.tflite', 'wb') as f:
    f.write(tflite_float32_optimized)
print(f"Modelo float32 optimizado: {len(tflite_float32_optimized)/1024:.2f} KB")

# Modelo int8 (cuantizado)
try:
    tflite_int8 = convert_to_tflite(optimal_model, quantize=True, optimize=True)
    with open('/content/drive/MyDrive/Tesis/Accelerometer_Dataset/optimal_mlp_int8.tflite', 'wb') as f:
        f.write(tflite_int8)
    print(f"Modelo int8: {len(tflite_int8)/1024:.2f} KB")
except Exception as e:
    print(f"Error en cuantización a int8: {e}")
    print("Continuando sin modelo int8...")

# Evaluar modelo TFLite float32
def evaluate_tflite_model(tflite_model_path, x_test, y_test):
    """Evalúa el modelo TFLite en datos de prueba"""
    # Cargar modelo TFLite
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Predicciones
    y_pred = []
    for i in range(len(x_test)):
        input_data = np.array(x_test[i:i+1], dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        y_pred.append(output_data[0])
    
    y_pred = np.array(y_pred)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    
    # Calcular precisión
    accuracy = np.mean(y_pred_classes == y_true_classes)
    
    return accuracy

# Evaluar modelo TFLite float32 optimizado
tflite_accuracy = evaluate_tflite_model(
    '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/optimal_mlp_float32_optimized.tflite',
    dataset['x_test'],
    dataset['y_test']
)

print(f"\nComparación de rendimiento:")
print(f"- Modelo Keras original: {optimal_test_acc:.4f}")
print(f"- Modelo TFLite optimizado: {tflite_accuracy:.4f}")
print(f"- Diferencia: {(tflite_accuracy - optimal_test_acc)*100:.2f}%")

print("\nModelos listos para ser usados en STM32CubeAI.")
print("Sigue estos pasos para implementar el modelo en tu STM32:")
print("1. Abre STM32CubeAI e importa el archivo TFLite (.tflite)")
print("2. Genera el código C para tu proyecto STM32")
print("3. Implementa las funciones de preprocesamiento necesarias")
print("4. Asegura la correcta asignación de memoria para las entradas/salidas del modelo")